<a href="https://colab.research.google.com/github/arijitlahiri20/LP-1-Practical-Practice/blob/master/HPC-1/HPC_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
code = """ #include<iostream>
#include<stdio.h>
#include<math.h>
#include<stdlib.h>
#include<time.h>
#define N 2048
using namespace std;

__global__ void Max(int *a)
{
    int tid = threadIdx.x;
    int step_size = 1;
    int num_of_threads = blockDim.x;
    
    while(num_of_threads>0){
        if(tid<num_of_threads){
            int first = tid*step_size*2;
            int second = first+step_size;
            if(a[second]>a[first])
                a[first]=a[second];                  
        }
        step_size<<=1;
        num_of_threads>>=1;
    }
    
}
__global__ void Min(int *a)
{
    int tid = threadIdx.x;
    int step_size = 1;
    int num_of_threads = blockDim.x;
    
    while(num_of_threads>0){
        if(tid<num_of_threads){
            int first = tid*step_size*2;
            int second = first+step_size;
            if(a[second]<a[first])
                a[first]=a[second];                  
        }
        step_size<<=1;
        num_of_threads>>=1;
    }
}
__global__ void sum(int *a)
{
    int tid = threadIdx.x;
    int step_size = 1;
    int num_of_threads = blockDim.x;
    
    while(num_of_threads>0){
        if(tid<num_of_threads){
            int first = tid*step_size*2;
            int second = first+step_size;
            a[first]+=a[second];                  
        }
        step_size<<=1;
        num_of_threads>>=1;
    }
}
__global__ void mean_diff(float *a,float mean)
{
    a[threadIdx.x]-=mean;
    a[threadIdx.x]*=a[threadIdx.x];
    
}
__global__ void _std(float *a,int n)
{
    int tid = threadIdx.x;
    int step_size = 1;
    int num_of_threads = blockDim.x;
    
    while(num_of_threads>0){
        if(tid<num_of_threads){
            int first = tid*step_size*2;
            int second = first+step_size;
            a[first]+=a[second];                  
        }
        step_size<<=1;
        num_of_threads>>=1;
    }
    a[0]/=a[0]/n;
}

int main()
{
    int *a = (int *)malloc(N*sizeof(int));
    float *f = (float *)malloc(N*sizeof(float));
    //srand(time(0));
    for(int i=0;i<N;i++){
        a[i]=rand()%100;
        f[i]=float(a[i]);
    } 
    int *a_cuda;
    float *f_cuda;          
    int answer;
    
    cudaMalloc((void **)&a_cuda,N*sizeof(int));
   
    //MAX
    cudaMemcpy(a_cuda,a,N*sizeof(int),cudaMemcpyHostToDevice);
    Max<<<1,N/2>>>(a_cuda);
    cudaMemcpy(&answer,a_cuda,sizeof(int),cudaMemcpyDeviceToHost);
    cout<<"Max : "<<answer<<endl;
    
    //MIN
    cudaMemcpy(a_cuda,a,N*sizeof(int),cudaMemcpyHostToDevice);
    Min<<<1,N/2>>>(a_cuda);
    cudaMemcpy(&answer,a_cuda,sizeof(int),cudaMemcpyDeviceToHost);
    cout<<"Min : "<<answer<<endl;
    
    //SUM
    cudaMemcpy(a_cuda,a,N*sizeof(int),cudaMemcpyHostToDevice);
    sum<<<1,N/2>>>(a_cuda);
    cudaMemcpy(&answer,a_cuda,sizeof(int),cudaMemcpyDeviceToHost);
    cout<<"Sum : "<<answer<<endl;
    
    //MEAN
    float mean = float(answer)/N;
    cout<<"Mean : "<<mean<<endl;
              
    cudaFree(a_cuda);
    cudaMalloc((void **)&f_cuda,N*sizeof(float));
    float result;
    //STD
    cudaMemcpy(f_cuda,f,N*sizeof(float),cudaMemcpyHostToDevice);          
    mean_diff<<<1,N>>>(f_cuda,mean);        
    _std<<<1,N/2>>>(f_cuda,N);
    cudaMemcpy(&result,f_cuda,sizeof(float),cudaMemcpyDeviceToHost);
    cout<<"Variance : "<<result<<endl;
    cout<<"Std. Dev. : "<<sqrt(result)<<endl;
              
    cudaFree(f_cuda);    
    cout<<endl;
    return 0;          
} """

In [0]:
text_file = open("code.cu", "w")
text_file.write(code)
text_file.close()

In [0]:
!nvcc code.cu

In [66]:
!./a.out

Max : 99
Min : 0
Sum : 68699
Mean : 33.5444
Variance : 2048
Std. Dev. : 45.2548



In [67]:
!nvprof ./a.out

==856== NVPROF is profiling process 856, command: ./a.out
Max : 99
Min : 0
Sum : 72797
Mean : 35.5454
Variance : 2048
Std. Dev. : 45.2548

==856== Profiling application: ./a.out
==856== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   20.01%  17.920us         1  17.920us  17.920us  17.920us  _std(float*, int)
                   18.19%  16.288us         1  16.288us  16.288us  16.288us  sum(int*)
                   17.61%  15.776us         1  15.776us  15.776us  15.776us  Max(int*)
                   17.51%  15.680us         1  15.680us  15.680us  15.680us  Min(int*)
                   15.97%  14.304us         4  3.5760us  3.3280us  4.0960us  [CUDA memcpy HtoD]
                   10.72%  9.6000us         4  2.4000us  2.3360us  2.5920us  [CUDA memcpy DtoH]
      API calls:   98.87%  126.14ms         2  63.069ms  161.47us  125.98ms  cudaMalloc
                    0.40%  508.27us         1  508.27us  508.27us  508.27us  c